# Part II: Model Development 3rd Pipeline 

In this part, we develop three unique pipelines for predicting backorder. We use the smart sample from Part I to fit and evaluate these pipelines. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools
import numpy as np
import pandas as pd

## Reload the smart sample here

In [ ]:

# Reload your smart sampling from local file 
# ----------------------------------

import joblib

# Load the sampled data from the file
hitesh_df = joblib.load('undersample_df.pkl')



## Normalize/standardize the data if required; otherwise ignore. You can perform this step inside the pipeline (if required). 

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()

# #Normalizse the dat 

# normalized_data = scaler.fit_transform(hitesh_df)
# normalized_data


In [ ]:
# #Checking if the data is normalised 

# import numpy as np

# normalized_data = scaler.fit_transform(hitesh_df)

# print("Mean:", np.mean(normalized_data, axis=0))
# print("Standard deviation:", np.std(normalized_data, axis=0)) #As mean is close to 0 and SD is close to 1 we can say that the data is normalized 



In [ ]:
# import pandas as pd

# normalized_df = pd.DataFrame(normalized_data, columns=hitesh_df.columns)

# normalized_df.describe()

## Split the data into Train/Test

In [ ]:
X = hitesh_df.iloc [:, :-1]
y = hitesh_df.went_on_backorder

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42, stratify=y)



In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn import svm
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest
from xgboost import XGBClassifier



### 3rd pipeline

In [ ]:
X_train3 = X_train
y_train3 = y_train

In [ ]:
# Add anomaly detection code  (Question #E209)
# ----------------------------------

svm = OneClassSVM(nu=0.08) # Set the desired contamination level
svm.fit(X_train3)

# Predict outliers
outliers_svm = svm.predict(X_train3) == -1 # Returns a boolean array with True for outliers

# Print number of outliers and the indices of the outliers
print(f"Number of outliers: {sum(outliers_svm)}")

X_svm = X_train3[~outliers_svm]
y_svm = y_train3[~outliers_svm]


In [ ]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E210)
# ----------------------------------

pipe3 = Pipeline ([
    ('scaler', MinMaxScaler()), 
    ('select', SelectKBest()), 
    ('xgb', XGBClassifier()) 
])


In [ ]:
param_grid3 = {
    'select__k': [5, 10, 15], # Number of top features to select
    'xgb__learning_rate': [0.1, 0.01, 0.001], # Learning rate for XGBoost
    'xgb__max_depth': [3, 5, 7], # Maximum depth of a tree
    'xgb__n_estimators': [50, 100, 200] # Number of trees to fit
}

In [ ]:
# Given an unbiased evaluation  (Question #E211)
# ----------------------------------

from sklearn.model_selection import GridSearchCV

model_grid3 = GridSearchCV(pipe3, param_grid3, cv = 5, n_jobs = 5)
model_grid3.fit(X_svm, y_svm)

